In [14]:
import onnx
import onnxruntime as ort
import numpy as np
from onnx import shape_inference
import sys
import torch
# Load the modified ONNX model
modified_model = onnx.load('init_networks/CNN_AE/newcnnae.onnx')

# Infer shapes
inferred_model = shape_inference.infer_shapes(modified_model)

# Save the inferred model (optional)
onnx.save(inferred_model, "inferred_model.onnx")

In [15]:
# Load the ONNX model
model_path = "init_networks/CNN_AE/newcnnae.onnx"
ort_sess  = ort.InferenceSession(model_path)


input_data  = np.random.rand(1, 36, 800).astype(np.float32)
# print(f"input data {noisy_test_data[:test_size]}")

# Run the model
output = ort_sess.run(None, {'input': input_data})

# 'output' contains the model's output (replace 'output_name' with the actual output name in your model)
print("Model output:", output[0].shape)
print("Model output:", output[0])

Model output: (1, 36, 800)
Model output: [[[ 0.02942611 -0.00596614  0.01723672 ...  0.01657646  0.01501433
    0.03952454]
  [ 0.0215117   0.02153412  0.02264793 ... -0.01843688  0.01669776
    0.01633377]
  [ 0.04966482  0.03233033  0.05889748 ...  0.06728861  0.03762051
    0.05749197]
  ...
  [ 0.00547374 -0.01206203 -0.0078625  ... -0.00027381 -0.01458102
    0.0115364 ]
  [-0.04439162 -0.071223   -0.08569658 ... -0.07923188 -0.05438021
   -0.06977148]
  [ 0.03640974  0.05703156  0.01793773 ...  0.07355741  0.02739896
    0.06127995]]]


In [16]:
from nntool.api import NNGraph
from nntool.api.utils import model_settings, quantization_options, tensor_plot
# import logging
# nntool_log = logging.getLogger('nntool')
# nntool_log.setLevel(logging.ERROR)

In [23]:
import os

# path = os.path.abspath ('init_networks/CNN_AE/newcnnae.onnx')
model = NNGraph.load_graph('init_networks/CNN_AE/newcnnae.onnx')
# model = NNGraph.load_graph('init_networks/fc_ae/fc_ae.onnx')
# model = NNGraph.load_graph('init_networks/tiny_cnn/tiny_tcn.onnx')
# Model show returns a table of information on the Graph
model.adjust_order()

# The equivalent of the fusions --scale8 command. The fusions method can be given a series of fusions to apply
# fusions('name1', 'name2', etc)
model.fusions('scaled_match_group')
print(model.show())

# Model draw can open or save a PDF with a visual representation of the graph
model.draw()

+------+-------------------+-------------------+------------+-------------+--------+--------+--------+---------+------------------+------------------+
| Step |     Step name     |     Operation     | Input Dims | Output Dims | Inputs | Active | Params |   Ops   |      Params      |      Hints       |
|      |                   |                   |  (0x1x2)   |   (0x1x2)   |        |  size  |  size  |         |                  |                  |
+======+===================+===================+============+=============+========+========+========+=========+==================+==================+
|  0   | input_1           | input             |  1x36x800  |  1x36x800   |        | 28800  |   0    |         | I 1x36x800       | in: none out:    |
|      |                   |                   |            |             |        |        |        |         |                  | none             |
+------+-------------------+-------------------+------------+-------------+--------+--------+-

/snap/core20/current/lib/x86_64-linux-gnu/libstdc++.so.6: version `GLIBCXX_3.4.29' not found (required by /lib/x86_64-linux-gnu/libproxy.so.1)
Failed to load module: /home/ahmad/snap/code/common/.cache/gio-modules/libgiolibproxy.so
Gtk-Message: 16:07:31.577: Failed to load module "gail"
Gtk-Message: 16:07:31.577: Failed to load module "atk-bridge"

(FoxitReader:22595): Gtk-WARNING **: 16:07:31.585: Unable to locate theme engine in module_path: "adwaita",

(FoxitReader:22595): Gtk-WARNING **: 16:07:31.586: Unable to locate theme engine in module_path: "pixmap",

(FoxitReader:22595): Gtk-WARNING **: 16:07:31.586: Unable to locate theme engine in module_path: "pixmap",

(FoxitReader:22595): Gtk-WARNING **: 16:07:31.587: Unable to locate theme engine in module_path: "pixmap",

(FoxitReader:22595): Gtk-WARNING **: 16:07:31.587: Unable to locate theme engine in module_path: "pixmap",

(FoxitReader:22595): Gtk-WARNING **: 16:07:31.587: Unable to locate theme engine in module_path: "pixmap",



In [24]:

model.fake_quantize(
    graph_options=quantization_options(
                use_ne16=True, hwc=True)
)

In [25]:
# s_model.adjust_order()
# s_model.fusions('scaled_match_group')
# 1.3 - buffersizes 
# AT_DEFAULTFLASH
# AT_MEM_L3_MRAMFLASH

l2_bf = 1300000 
# l2_bf= 1300000 - 31744

res = model.execute_on_target(at_log=True, settings=model_settings(
            l1_size=120000,
            l2_size=l2_bf, graph_const_exec_from_flash=True, l3_flash_device="AT_MEM_L3_DEFAULTFLASH"), dont_run=False, performance=True, print_output=True, directory='/tmp/unsandonedge', at_loglevel=1)

Script started, output log file is '/tmp/tmp_ds0bo2p/log.txt'.
-- [Custom BSP] Custom BSP unused.
-- [Python] Interpreter found here /home/ahmad/envs/gwt_tools/bin/python3.10
-- [Kconfig] KCONFIG_CONFIG environnement variable has not been exported.
-- [Kconfig] Using default value sdk: "sdk.config".
-- [AutoTiler] Tiler lib path: /home/ahmad/Desktop/gap_sdk_private/tools/autotiler_v3/Autotiler/LibTile.a
-- [NNTOOL] Using nntool in /home/ahmad/Desktop/gap_sdk_private/tools/nntool/scripts/nntool
-- [SFU] Using SFU in 
-- [ CMAKE_C_FLAGS ]  -march=rv32imcxgap9 -mPE=8 -mFC=1 -mint64  -fcommon -fno-jump-tables -fno-tree-loop-distribute-patterns -fno-delete-null-pointer-checks -fomit-frame-pointer -Os -fmessage-length=0 -fno-exceptions -ffunction-sections -fdata-sections -funsigned-char -Wall -Wextra -Werror -Wno-unused-parameter -Wno-unused-function -Wno-unused-variable -Wno-unused-but-set-variable -Wno-implicit-fallthrough  -Wno-discarded-qualifiers
-- [CMAKE_CXX_FLAGS]  -march=rv32imcxgap